In [1]:
import Pkg
Pkg.activate("packages")
Pkg.instantiate()
using CSV, DataFrames, Plots, Plots.PlotMeasures, StatsPlots

  Activating environment at `~/Documents/Still evolving/Quantifying-SIM/packages/Project.toml`
Precompiling project...
  ✓ PrecompileTools
  ✓ Wayland_protocols_jll
  ✓ Xorg_libpthread_stubs_jll
  ✓ IntelOpenMP_jll
  ✓ Xorg_xtrans_jll
  ✓ Zstd_jll
  ✓ Xorg_libXdmcp_jll
  ✓ Xorg_libXau_jll
  ✓ Rmath_jll
  ✓ Libiconv_jll
  ✓ LLVMOpenMP_jll
  ✓ JpegTurbo_jll
  ✓ OpenSSL_jll
  ✓ Expat_jll
  ✓ FreeType2_jll
  ✓ MKL_jll
  ✓ Rmath
  ✓ XML2_jll
  ✓ Pixman_jll
  ✓ RecipesBase
  ✓ Libtiff_jll
  ✓ Wayland_jll
  ✓ Fontconfig_jll
  ✓ XSLT_jll
  ✓ OpenSSL
  ✓ Gettext_jll
  ✓ Xorg_libxcb_jll
  ✓ Glib_jll
  ✓ Xorg_xcb_util_jll
  ✓ Xorg_libX11_jll
  ✓ FFTW
  ✓ SpecialFunctions
  ✓ Xorg_xcb_util_renderutil_jll
  ✓ Xorg_xcb_util_keysyms_jll
  ✓ Xorg_xcb_util_wm_jll
  ✓ Xorg_xcb_util_image_jll
  ✓ Xorg_libXext_jll
  ✓ Xorg_libXrender_jll
  ✓ Xorg_libxkbfile_jll
  ✓ Xorg_libXfixes_jll
  ✓ DualNumbers
  ✓ HTTP
  ✓ DiffRules
  ✓ Xorg_libXinerama_jll
  ✓ Libglvnd_jll
  ✓ Xorg_libXrandr_jll
  ✓ Xorg_xkbcomp_jl

LoadError: Failed to precompile CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b] to /Users/s2089131/.julia/compiled/v1.6/CSV/jl_jOMmq0.